# Some Protocols for Fair Division

In [4]:
from problem import Problem
import fairnessMeasures
#import simulations

Several protocols have been implemented. They can be accessed by importing the module protocols. 

In [5]:
import protocols

## 4.1 Adjusted Winner

In [6]:
p1 = Problem(3,4,'uniform',centralized=True)
print(p1)

agent 0{'r2': 53, 'r1': 15, 'r0': 93, 'r3': 83}
agent 1{'r2': 88, 'r1': 89, 'r0': 77, 'r3': 55}
agent 2{'r2': 59, 'r1': 17, 'r0': 27, 'r3': 36}



In [7]:
protocols.adjustedWinner(p1,verbose=True)

Allocation phase:
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
agent  0           ['r2', 'r1', 'r0', 'r3']	244
agent  1           ['r2', 'r1', 'r0', 'r3']	309
agent  2                                 []	 0

[(1.492, 'r2'), (1.528, 'r3'), (2.852, 'r0'), (5.235, 'r1')]
Resource  r2  moves from  1  to  2
Resource  r3  moves from  1  to  2
Resource  r0  moves from  1  to  2
Resource  r0  will be splitted!
Agent  2  gets  0.317  of resource  r0
Both agents get utility: 130.559


Make sure that you understand exactly why items are allocated this way. 

### Manipulating Adjusted Winner

In [8]:
p2 = Problem(3,2,'uniform',centralized=True)
p2.setUtilities(
[{'r0':0,'r1':0},\
{'r0':25,'r1':75},\
{'r0':75,'r1':25}]
)
print (p2)

agent 0{'r1': 0, 'r0': 0}
agent 1{'r1': 75, 'r0': 25}
agent 2{'r1': 25, 'r0': 75}



Find a manipulation for agent 1, that is, a way to misrepresent the preference of the agent such that the utility is higher. Note that you will need to compute the allocation with the declared preferences, but that the actual utility enjoyed by agents must be computed with their 'true' preferences. 

What is the "best" manipulation that agent 1 can do? 
To evaluate this, it will be useful to run a script trying all the different values possibly announced by agent 1, and to plot the utility obtained with each of these. 

### Adjusted Winner for More than Two Agents

Adjusted Winner is just defined for two agents. Could you try to come up with a version fitting more than two agents? 
Taking inspiration from the code for 2 agents, could you implement and test your version? 

## 4. 2 Picking Sequences

In [9]:
p3 = Problem(4,6,'empty', centralized=True)
p3.setUtilities(
[{'r0':0,'r1':0,'r2':0,'r3':0,'r4':0,'r5':0},\
{'r0':1,'r1':2,'r2':5,'r3':3,'r4':7,'r5':2},\
{'r0':2,'r1':6,'r2':8,'r3':1,'r4':1,'r5':2},\
{'r0':5,'r1':4,'r2':4,'r3':3,'r4':2,'r5':2}]
)
print (p3)
print (p3.printAllocation())

agent 0{'r2': 0, 'r1': 0, 'r5': 0, 'r0': 0, 'r4': 0, 'r3': 0}
agent 1{'r2': 5, 'r1': 2, 'r5': 2, 'r0': 1, 'r4': 7, 'r3': 3}
agent 2{'r2': 8, 'r1': 6, 'r5': 2, 'r0': 2, 'r4': 1, 'r3': 1}
agent 3{'r2': 4, 'r1': 4, 'r5': 2, 'r0': 5, 'r4': 2, 'r3': 3}

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
agent  0['r2', 'r1', 'r5', 'r0', 'r4', 'r3']	 0
agent  1                                 []	 0
agent  2                                 []	 0
agent  3                                 []	 0



Let us apply a picking sequence on our problem p3.

In [10]:
s0 = [1,2,3,2,3,1]
protocols.pickingSequence(p3,s0,verbose=True)

agent  1  picks  r4
agent  2  picks  r2
agent  3  picks  r0
agent  2  picks  r1
agent  3  picks  r3
agent  1  picks  r5


In [11]:
print(p3.printAllocation())

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
agent  0                                 []	 0
agent  1                       ['r4', 'r5']	 9
agent  2                       ['r2', 'r1']	14
agent  3                       ['r0', 'r3']	 8



In [26]:
print(fairnessMeasures.envyMatrix(p3))

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


It is also possible to generate standard sequences, like balanced or alternate ones. 

In [20]:
s= protocols.generateSequence(3,6,'balanced')
print(s)

[1, 2, 3, 3, 2, 1]


### What is the fairest picking sequence?

In [21]:
import simulations

Let us consider 3 agents and 5 items. Can you propose some sequence which would do well in terms of egalitarian social welfare? You can simulate a number of picking sequences by specifying: the number of experiments, the number of agents (remember to count agent 0 here-to be fixed sorry), the number of objects, the sequence, and the ways utilities are generated.
    

In [30]:
simulations.simulationPickingSequences(1000,4,5,[1,1,2,2,3],'borda',verbose=False) # to start with a bad sequence

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
= Number of experiments:                 1000
= Average egalitarian sw:                 2.986
= Ratio of proportional:                 0.19
= Ratio of envy free:                    0.057
= Average number of envious:             1.145
= Average max envy:                       4.42
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=



### And to conclude: 
For 3 agents, and 6 and 8 objects, could you find the fairest picking sequences in terms of: 
* egalitarian social welfare
* average max envy

In [29]:
simulations.simulationPickingSequences(1000,4,6,[1,1,3,2,2,3],'borda',verbose=False) # to start with a bad sequence

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
= Number of experiments:                 1000
= Average egalitarian sw:                 7.625
= Ratio of proportional:                 0.79
= Ratio of envy free:                    0.489
= Average number of envious:             0.585
= Average max envy:                      1.461
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=



### References

* The discussion and example about Adjusted Winner Manipulation is taken from a video by Eric Pacuit: 
https://www.youtube.com/watch?v=RtcnSXL69NQ

* See (Bouveret and Lang, IJCAI-11) for more details about picking sequences. 